In [1]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('max_seq_item', None)

In [2]:
df_sen_pro_act = pd.read_csv('senators_signed_pro_act.csv')
df_google = pd.read_csv('google_donations.csv')

In [3]:
df_sen_pro_act['last_name'] = df_sen_pro_act.senators.str.split('Sen. ', expand=True)[1].str.split(',', expand=True)[0].str.upper()
df_sen_pro_act.head()

,senators,last_name
0,"Sen. Murray, Patricia [D-WA]",MURRAY
1,"Sen. Schumer, Charles E. [D-NY]*",SCHUMER
2,"Sen. Blumenthal, Richard [D-CT]*",BLUMENTHAL
3,"Sen. Duckworth, Tammy [D-IL]*",DUCKWORTH
4,"Sen. Booker, Cory A. [D-NJ]*",BOOKER


In [4]:
df_sen_pro_act['first_name'] = df_sen_pro_act.senators.str.split('Sen. ', expand=True)[1].str.split(',', expand=True)[1].str.split('[',expand=True)[0].str.upper()
df_sen_pro_act.head()

,senators,last_name,first_name
0,"Sen. Murray, Patricia [D-WA]",MURRAY,PATRICIA
1,"Sen. Schumer, Charles E. [D-NY]*",SCHUMER,CHARLES E.
2,"Sen. Blumenthal, Richard [D-CT]*",BLUMENTHAL,RICHARD
3,"Sen. Duckworth, Tammy [D-IL]*",DUCKWORTH,TAMMY
4,"Sen. Booker, Cory A. [D-NJ]*",BOOKER,CORY A.


In [5]:
df_sen_pro_act['first_name'] = df_sen_pro_act['first_name'].apply(lambda x: x.split()[0])
df_sen_pro_act.head()

,senators,last_name,first_name
0,"Sen. Murray, Patricia [D-WA]",MURRAY,PATRICIA
1,"Sen. Schumer, Charles E. [D-NY]*",SCHUMER,CHARLES
2,"Sen. Blumenthal, Richard [D-CT]*",BLUMENTHAL,RICHARD
3,"Sen. Duckworth, Tammy [D-IL]*",DUCKWORTH,TAMMY
4,"Sen. Booker, Cory A. [D-NJ]*",BOOKER,CORY


In [6]:
df_google[['disbursement_amount', 'candidate_name', 'candidate_last_name', 'candidate_first_name', 'candidate_office']]

,disbursement_amount,candidate_name,candidate_last_name,candidate_first_name,candidate_office
0,-5000,"LUMMIS, CYNTHIA",LUMMIS,CYNTHIA,S
1,-5000,"YOUNG, TODD",YOUNG,TODD,S
2,-5000,"SALAZAR, MARIA ELVIRA",SALAZAR,MARIA ELVIRA,H
3,-5000,NaN,NaN,NaN,NaN
4,-4000,"MERKLEY, JEFF",MERKLEY,JEFF,S
5,-4000,"YOUNG, TODD",YOUNG,TODD,S
6,-2500,"CRENSHAW, DAN",CRENSHAW,DAN,H
7,-2500,NaN,NaN,NaN,NaN
8,-2500,"SPARTZ, VICTORIA",SPARTZ,VICTORIA,H
9,-2500,"PADILLA, ALEX",PADILLA,ALEX,S


In [8]:
merged_df = df_google.merge(df_sen_pro_act.set_index(['last_name', 'first_name']), 
                      left_on=['candidate_last_name', 'candidate_first_name'], 
                      right_index=True, 
                      how='left')

In [11]:
merged_df_1 = merged_df[merged_df['senators'].notna()]
sen_pro_act_and_google_money = merged_df_1[['disbursement_amount', 'candidate_name', 'candidate_last_name', 'candidate_first_name', 'candidate_office', 'senators']]

In [12]:
sen_pro_act_and_google_money = sen_pro_act_and_google_money.groupby('senators').sum('disbursement_amount').sort_values(by='disbursement_amount', ascending=True).reset_index()

In [13]:
duckworth = pd.DataFrame({'senators': ['Sen. Duckworth, Tammy [D-IL]*'], 'disbursement_amount': ['7500.0']})

# concatenate the two dataframes along the axis=0
sen_pro_act_and_google_money_2 = pd.concat([sen_pro_act_and_google_money, duckworth], axis=0, ignore_index=True)

sen_pro_act_and_google_money = sen_pro_act_and_google_money_2
sen_pro_act_and_google_money

,senators,disbursement_amount
0,"Sen. Merkley, Jeff [D-OR]*",-4000
1,"Sen. Kaine, Tim [D-VA]*",1000
2,"Sen. Leahy, Patrick J. [D-VT]*",1000
3,"Sen. Heinrich, Martin [D-NM]*",2000
4,"Sen. King, Angus S., Jr. [I-ME]",2000
5,"Sen. Peters, Gary C. [D-MI]*",2000
6,"Sen. Rosen, Jacky [D-NV]*",2000
7,"Sen. Stabenow, Debbie [D-MI]*",2000
8,"Sen. Duckworth, Tammy [D-IL]*",2500
9,"Sen. Tester, Jon [D-MT]*",3000


In [14]:
sen_pro_act_and_google_money['disbursement_amount'] = sen_pro_act_and_google_money['disbursement_amount'].astype(float)

In [20]:
sen_pro_act_and_google_money.to_csv('sen_pro_act_and_google_money.csv', index=False)

In [21]:
import os

directory = '/Users/rachelp/Downloads/lawmakers_pro_act_google_money'
file_list = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]


In [22]:
google_data = pd.DataFrame()

In [23]:
for file_name in file_list:
    data = pd.read_csv(file_name)
    google_data = pd.concat([google_data, data], ignore_index=True)

In [24]:
google_data.to_csv('google_data.csv', index=False)